# 【問題1】スクラッチを振り返る

ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙してください。<br>
（例）重みを初期化する必要があった<br>
     エポックのループが必要だった<br>
それらがフレームワークにおいてはどのように実装されるかを今回覚えていきましょう。<br>


- ミニバッチで学習を回す
- フォワードプロ、バックプロでW,Bの更新
- LOSS値の算出
- 学習率他、ハイパーパラメーターの設定
- 畳み込みのチャネル、フィルターの設計

# データセットの用意

# 【問題2】スクラッチとTensorFlowの対応を考える

以下のサンプルコードを見て、先ほど列挙した「ディープラーニングを実装するために必要なもの」が<br>
TensorFlowではどう実装されているかを確認してください。<br>
それを簡単に言葉でまとめてください。単純な一対一の対応であるとは限りません。

- バックプロパゲーションの計算式がない
　optimizer.minimize(loss_op)で計算？
- 交差エントロピーの計算自体はあるが計算式がない（推定結果を正解値の入力するだけ）
- 出力結果はprobaで出力

In [199]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み

iris_data = load_iris()
X = iris_data.data[50:]
y = iris_data.target[50:] - 1

# dataset_path ="Iris.csv"
# df = pd.read_csv(dataset_path)
# # データフレームから条件抽出
# df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
# y = df["Species"]
# X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
# y = np.array(y)
# X = np.array(X)
# # ラベルを数値に変換
# y[y=='Iris-versicolor'] = 0
# y[y=='Iris-virginica'] = 1
y = y.astype(np.int)[:, np.newaxis]

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01 #学習率
batch_size = 10 #バッチサイズ
num_epochs = 10 #エポック数
n_hidden1 = 50 #２層目
n_hidden2 = 100 #１層目
n_input = X_train.shape[1] #特徴量
n_samples = X_train.shape[0] #サンプル数
n_classes = 1 #２値分類（１個のワンホット）
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input]) #（サンプル数未定、特徴量数）
Y = tf.placeholder("float", [None, n_classes]) #（サンプル数未定、クラス分類）
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size) #ミニバッチでデータを分割
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])), #重み（４、５０）
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])), #重み（５０、１００）
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes])) #重み（１００、３）
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])), #バイアス（５０）
        'b2': tf.Variable(tf.random_normal([n_hidden2])),#バイアス（１００）
        'b3': tf.Variable(tf.random_normal([n_classes]))#バイアス（３）
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1']) #X@W1 + b1
    layer_1 = tf.nn.relu(layer_1) #reru
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2']) #X@W2 + b2
    layer_2 = tf.nn.relu(layer_2) #reru
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] #X@W3 + b3
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X) #上記クラスの計算
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 出力結果と正解値で交差エントロピー
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)#アダムグラッド？
train_op = optimizer.minimize(loss_op)#クロスエントロピーの結果からバックプロパゲーション？
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))#正解、不正解のBoolを返す
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))#１、０に変換し正解率を計算
# variableの初期化
init = tf.global_variables_initializer()#W,Bの初期化

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)#初期化
    for epoch in range(num_epochs):#エポック数分最適化を繰り返す
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int) #小数点以下切り下げ？でバッチのループ数計算
        total_loss = 0 #ロスを初期化
        total_acc = 0 #正解率を初期化
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):#ミニバッチの計算
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y}) 
            #最適化の数値としてtrain_opを指定、引数として辞書型のミニバッチ（）
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #ロスエントロピーとアキュラシーをX,Yに対して計算
            total_loss += loss #ロスをトータルロスに追加 
            total_acc += acc #アキュラシーをトータルアキュラシーに追加
        total_loss /= n_samples #サンプル数で割る
        total_acc /= n_samples #サンプル数で割る
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})#バリデーションでバルロスを計算
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
        # エポックごとにロス、バルロス、正解率をプリント
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})#テストデータで正解率を計算
    print("test_acc : {:.3f}".format(test_acc))

(100, 1)
Epoch 0, loss : 28.8249, val_loss : 62.5707, acc : 0.750, val_acc : 0.375
Epoch 1, loss : 27.4720, val_loss : 59.5359, acc : 0.750, val_acc : 0.375
Epoch 2, loss : 57.6751, val_loss : 28.2467, acc : 0.250, val_acc : 0.625
Epoch 3, loss : 5.4079, val_loss : 13.2653, acc : 0.750, val_acc : 0.375
Epoch 4, loss : 6.1407, val_loss : 9.5193, acc : 0.250, val_acc : 0.625
Epoch 5, loss : 0.4262, val_loss : 4.7512, acc : 0.750, val_acc : 0.625
Epoch 6, loss : 1.4025, val_loss : 6.9663, acc : 0.750, val_acc : 0.688
Epoch 7, loss : 0.0187, val_loss : 3.9771, acc : 1.000, val_acc : 0.625
Epoch 8, loss : 0.3151, val_loss : 5.9727, acc : 0.750, val_acc : 0.688
Epoch 9, loss : 0.0065, val_loss : 3.9492, acc : 1.000, val_acc : 0.625
test_acc : 0.800


# 他のデータセットへの適用
- Iris（3種類全ての目的変数を使用）
- House Prices
- どのデータセットもtrain, val, testの3種類に分けて使用してください。

# 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成

In [237]:
# データセットの読み込み
iris_data = load_iris()
X = iris_data.data[:]
y = iris_data.target[:]
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_one_hot = enc.fit_transform(y[:, np.newaxis])
y = y_one_hot.astype(np.int)
# print(y.shape)
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

np.argmax(y_train,1)

X_test.shape

(30, 4)

In [238]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

# ハイパーパラメータの設定
learning_rate = 0.002 #学習率
batch_size = 10 #バッチサイズ
num_epochs = 10 #エポック数
n_hidden1 = 50 #２層目
n_hidden2 = 100 #１層目
n_input = X_train.shape[1] #特徴量
n_samples = X_train.shape[0] #サンプル数
n_classes = 3 #２値分類（１個のワンホット）
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input]) #（サンプル数未定、特徴量数）
Y = tf.placeholder("float", [None, n_classes]) #（サンプル数未定、クラス分類）
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size) #ミニバッチでデータを分割
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])), #重み（４、５０）
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])), #重み（５０、１００）
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes])) #重み（１００、３）
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])), #バイアス（５０）
        'b2': tf.Variable(tf.random_normal([n_hidden2])),#バイアス（１００）
        'b3': tf.Variable(tf.random_normal([n_classes]))#バイアス（３）
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1']) #X@W1 + b1
    layer_1 = tf.nn.relu(layer_1) #reru
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2']) #X@W2 + b2
    layer_2 = tf.nn.relu(layer_2) #reru
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] #X@W3 + b3
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X) #上記クラスの計算
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# 出力結果と正解値で交差エントロピー
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)#アダムグラッド？
train_op = optimizer.minimize(loss_op)#クロスエントロピーの結果からバックプロパゲーション？
# 推定結果
correct_pred = tf.equal(tf.argmax(Y,1),tf.argmax(logits,1))#正解、不正解のBoolを返す
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))#１、０に変換し正解率を計算
# variableの初期化
init = tf.global_variables_initializer()#W,Bの初期化

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)#初期化
    for epoch in range(num_epochs):#エポック数分最適化を繰り返す
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int) #小数点以下切り下げ？でバッチのループ数計算
        total_loss = 0 #ロスを初期化
        total_acc = 0 #正解率を初期化
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):#ミニバッチの計算
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y}) 
            #最適化の数値としてtrain_opを指定、引数として辞書型のミニバッチ（）
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #ロスエントロピーとアキュラシーをX,Yに対して計算
            total_loss += loss #ロスをトータルロスに追加 
            total_acc += acc #アキュラシーをトータルアキュラシーに追加
        total_loss /= n_samples #サンプル数で割る
        total_acc /= n_samples #サンプル数で割る
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})#バリデーションでバルロスを計算
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
        # エポックごとにロス、バルロス、正解率をプリント
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})#テストデータで正解率を計算
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 82.9003, val_loss : 45.5487, acc : 0.333, val_acc : 0.542
Epoch 1, loss : 7.8293, val_loss : 10.5941, acc : 0.833, val_acc : 0.417
Epoch 2, loss : 2.4431, val_loss : 6.6046, acc : 0.500, val_acc : 0.667
Epoch 3, loss : 0.4123, val_loss : 4.5080, acc : 0.667, val_acc : 0.750
Epoch 4, loss : 0.1229, val_loss : 4.7169, acc : 0.833, val_acc : 0.792
Epoch 5, loss : 0.0064, val_loss : 3.9906, acc : 1.000, val_acc : 0.833
Epoch 6, loss : 0.0061, val_loss : 3.4577, acc : 1.000, val_acc : 0.875
Epoch 7, loss : 0.0093, val_loss : 3.2081, acc : 1.000, val_acc : 0.875
Epoch 8, loss : 0.0089, val_loss : 3.0325, acc : 1.000, val_acc : 0.875
Epoch 9, loss : 0.0051, val_loss : 3.0715, acc : 1.000, val_acc : 0.875
test_acc : 0.933


# 【問題4】House Pricesのモデルを作成

In [240]:
df = pd.read_csv('/Users/suzukiyuuta/Downloads/train.csv')#教師データを読み込む
df_X_ori = np.log(df.loc[:,['GrLivArea','YearBuilt']].values)
df_Y_ori = np.log(df.loc[:,['SalePrice']].values)

df_X_ori.shape
df_Y_ori

array([[12.24769432],
       [12.10901093],
       [12.31716669],
       ...,
       [12.49312952],
       [11.86446223],
       [11.90158345]])

In [241]:
X_train.shape[1]

2

In [242]:
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(df_X_ori, df_Y_ori, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.001 #学習率
batch_size = 10 #バッチサイズ
num_epochs = 10 #エポック数
n_hidden1 = 50 #２層目
n_hidden2 = 100 #１層目
n_input = X_train.shape[1] #特徴量
n_samples = X_train.shape[0] #サンプル数
n_classes = 1 #２値分類（１個のワンホット）
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input]) #（サンプル数未定、特徴量数）
Y = tf.placeholder("float", [None, n_classes]) #（サンプル数未定、クラス分類）
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size) #ミニバッチでデータを分割
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])), #重み（４、５０）
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])), #重み（５０、１００）
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes])) #重み（１００、３）
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])), #バイアス（５０）
        'b2': tf.Variable(tf.random_normal([n_hidden2])),#バイアス（１００）
        'b3': tf.Variable(tf.random_normal([n_classes]))#バイアス（３）
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1']) #X@W1 + b1
    layer_1 = tf.nn.relu(layer_1) #reru
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2']) #X@W2 + b2
    layer_2 = tf.nn.relu(layer_2) #reru
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] #X@W3 + b3
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X) #上記クラスの計算
# 目的関数
loss_op = tf.reduce_sum(tf.math.square(Y - logits))
# 出力結果と正解値で交差エントロピー
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)#アダムグラッド？
train_op = optimizer.minimize(loss_op)#クロスエントロピーの結果からバックプロパゲーション？
# 推定結果
# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))#正解、不正解のBoolを返す
# 指標値計算
accuracy = tf.reduce_sum(tf.math.square(Y - logits))#１、０に変換し正解率を計算
# variableの初期化
init = tf.global_variables_initializer()#W,Bの初期化

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)#初期化
    for epoch in range(num_epochs):#エポック数分最適化を繰り返す
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int) #小数点以下切り下げ？でバッチのループ数計算
        total_loss = 0 #ロスを初期化
        total_acc = 0 #正解率を初期化
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):#ミニバッチの計算
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y}) 
            #最適化の数値としてtrain_opを指定、引数として辞書型のミニバッチ（）
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            #ロスエントロピーとアキュラシーをX,Yに対して計算
            total_loss += loss #ロスをトータルロスに追加 
            total_acc += acc #アキュラシーをトータルアキュラシーに追加
        total_loss /= n_samples #サンプル数で割る
        total_acc /= n_samples #サンプル数で割る
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})#バリデーションでバルロスを計算
#         print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}".format(epoch, loss, val_loss,))
        # エポックごとにロス、バルロス、正解率をプリント
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})#テストデータで正解率を計算
    print("test_MSE : {:.3f}".format(test_acc))

Epoch 0, loss : 19.4198, val_loss : 1382.9233
Epoch 1, loss : 3.8362, val_loss : 501.0757
Epoch 2, loss : 1.2261, val_loss : 50.7540
Epoch 3, loss : 0.7092, val_loss : 39.9238
Epoch 4, loss : 0.4768, val_loss : 35.1797
Epoch 5, loss : 0.4551, val_loss : 32.3300
Epoch 6, loss : 0.5303, val_loss : 31.0605
Epoch 7, loss : 0.5842, val_loss : 30.3951
Epoch 8, loss : 0.6696, val_loss : 29.5379
Epoch 9, loss : 0.6450, val_loss : 28.9352
test_MSE : 52.161


# 【問題5】MNISTのモデルを作成

In [105]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

Using TensorFlow backend.


In [106]:
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
print(y_train.shape) # (60000,)
print(y_train_one_hot.shape) # (60000, 10)
print(y_train_one_hot.dtype) # float64

1.0
0.0
(60000,)
(60000, 10)
float64


In [107]:
X_train_0, X_val_0, y_train_0, y_val_0 = train_test_split(X_train, y_train_one_hot, test_size=0.99)

X_train_0 = X_train_0[:, np.newaxis, :, :]
X_val_0 = X_val_0[:, np.newaxis, :, :]

print(X_train_0.shape) # (600, 28,28,1)
print(X_val_0.shape) # (59400, 28,28,1)
print(y_train_0.shape) # (600, 10)
print(y_val_0.shape) # (59400, 10)

(600, 1, 28, 28)
(59400, 1, 28, 28)
(600, 10)
(59400, 10)


In [153]:
# learning_rate = 0.1 #学習率
# batch_size = 10 #バッチサイズ
# num_epochs = 20 #エポック数
# n_hidden1 = 50 #２層目
# n_hidden2 = 100 #１層目
# n_input = X_train.shape[1] #特徴量
# n_samples = X_train.shape[0] #サンプル数
# n_classes = 10 #回帰のため１



mnist = input_data.read_data_sets("mnist/", one_hot=True)

# print(mnist.test.images.shape)
# print(mnist.test.labels.shape)

pixels = 28
nums = 10

# 計算グラフに渡す引数の形を決める
X = tf.placeholder(tf.float32, shape=(None,pixels), name="x") #（サンプル数未定、特徴量数）
Y = tf.placeholder(tf.float32, shape=(None,nums), name="y_") #（サンプル数未定、クラス分類）

def weight_variable(name,shape):
    W_init = tf.truncated_normal(shape, stddev=0.1)
    W = tf.Variable(W_init, name="W"+name)
    return W
def bias_variable(name,size):
    b_init = tf.constant(0.1, shape=[size])
    b = tf.Variable(b_init, name="b_"+name)
    return b

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME")
def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

with tf.name_scope("conv1")as scope:
    W_conv1 = weight_variable("conv1",[5,5,1,32])
    b_conv1 = bias_variable("conv1",32)
    x_image = tf.reshape(x,[-1,28,28,1])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

with tf.name_scope("pool1")as scope:
    h_pool1 = max_pool(h_conv1)

with tf.name_scope("conv2")as scope:
    W_conv2 = weight_variable("conv2",[5,5,32,64])
    b_conv2 = bias_variable("conv2",64)
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    
with tf.name_scope("pool2")as scope:
    h_pool2 = max_pool(h_conv2)

with tf.name_scope("fully_connected")as scope:
    n = 7*7*64
    W_fc = weight_variable("fc",[n,1024])
    b_fc = bias_variable("fc",1024)
    h_pool2_flat = tf.reshape(h_pool2,[-1,n])
    h_fc = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc) + b_fc)
    
with tf.name_scope("dropout")as scope:
    keep_prob = tf.placeholder(tf.float32)
    h_fc_drop = tf.nn.dropout(h_fc, keep_prob)
    
with tf.name_scope("readout")as scope:
    W_fc2 = weight_variable("fc2",[1024,10])
    b_fc2 = bias_variable("fc2",10)
    y_conv = tf.nn.softmax(tf.matmul(h_fc_drop, W_fc2) + b_fc2)

with tf.name_scope("loss")as scope:
    cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))

with tf.name_scope("training")as scope:
    optimizer = tf.train.AdamOptimizer(0.0001)
    train_step = optimizer.minimize(cross_entropy)

with tf.name_scope("predict")as scope:
    predict_step = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
    accuracy_step = tf.reduce_mean(tf.cast(predict_step,tf.float32))

def set_feed(images, labels, prob):
    return{x: images, y_: labels, keep_prob: prob}

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    tw = tf.summary.FileWriter('log_dir',graph=sess.graph)
    test_fd = set_feed(mnist.test.images, mnist.test.labels,1)
    for step in range(1000):
        batch = mnist.train.next_batch(50)
        fd = set_feed(batch[0],batch[1],0.5)
        _,loss = sess.run([train_step, cross_entropy], feed_dict=fd)
        if step % 100 == 0:
            acc = sess.run(accuracy_step, feed_dict=test_fd)
            print("step=", step,"loss=",loss,"acc=",acc)
    acc = sess.run(accuracy_step, feed_dict=test_fd)
    print("正解率=",acc)
    

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz
step= 0 loss= 480.89835 acc= 0.1064
step= 100 loss= 50.972904 acc= 0.8586
step= 200 loss= 24.164627 acc= 0.9121
step= 300 loss= 26.35611 acc= 0.9306
step= 400 loss= 34.81744 acc= 0.9384
step= 500 loss= 9.854622 acc= 0.9417
step= 600 loss= 23.888836 acc= 0.9505
step= 700 loss= 18.001362 acc= 0.9503
step= 800 loss= 10.534403 acc= 0.954
step= 900 loss= 8.4989395 acc= 0.9623
正解率= 0.9648


In [154]:

# 画像データ
x = tf.placeholder("float", [None, 784])

# モデルの重み
W = tf.Variable(tf.zeros([784, 10]))

# モデルのバイアス
b = tf.Variable(tf.zeros([10]))

# トレーニングデータxとモデルの重みWを乗算した後、モデルのバイアスbを足し、
# ソフトマックス回帰（ソフトマックス関数）を適用
y = tf.nn.softmax(tf.matmul(x, W) + b)

# 正解データ
y_ = tf.placeholder("float", [None, 10])

# 損失関数をクロスエントロピーとする
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))

# 学習係数を0.01として、勾配降下アルゴリズムを使用して、
# クロスエントロピーを最小化する
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

# 変数の初期化
init = tf.initialize_all_variables()

# セッションの作成
sess = tf.Session()

# セッションの開始および初期化の実行
sess.run(init)

# トレーニングの開始
for i in range(1000):
    # トレーニングデータからランダムに100個抽出する
    batch_xs, batch_ys = mnist.train.next_batch(100)

    # 確率的勾配降下によりクロスエントロピーを最小化するよう重みを更新
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

# 予測値と正解値を比較して、bool値（true or false）にする
# tf.argmax(y, 1)は、予測値の各行で、最大値となるインデックスを一つ返す
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

# bool値を0 or 1に変換して平均値をとる -> 正解率
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# テストデータを与えて、テストデータの正解率の表示
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9139


In [155]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

# Mnistに使うデータセットをインポートする
mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data',
  one_hot=True)

# グラフの作成
x = tf.placeholder(tf.float32, [None, 784]) # 入力するPlaceholder
W = tf.Variable(tf.zeros([784, 10])) # 重み
b = tf.Variable(tf.zeros([10])) # バイアス
y = tf.matmul(x, W) + b #内積計算とバイアスの加算

# 正解ラベル用のplaceholeder
y_ = tf.placeholder(tf.float32, [None, 10])

# 損失の計算方法と、オプティマイザーを定義
cross_entropy = tf.reduce_mean(
 tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# セッションの作成と初期化
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()
 
# 学習部(1000回学習)
for _ in range(1000):
 batch_xs, batch_ys = mnist.train.next_batch(100)
 sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

# テスト部
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images,
 y_: mnist.test.labels}))

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz


/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


0.9168


In [156]:
# データのインポート
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
 
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
sess = tf.InteractiveSession()
 
# モデルの作成
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
 
# 損失とオプティマイザーを定義
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = 　　　　　　　　　　　　
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
 
# 訓練
tf.initialize_all_variables().run()
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  train_step.run({x: batch_xs, y_: batch_ys})
 
# 訓練モデルのテスト
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x: mnist.test.images, y_: mnist.test.labels}))

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
0.9159
